# Feature store example
---

The idea of this notebook is to give a example on how we manage:
- Ingest data in a batch way
- Retrieve data from the feature store for training and inference mode.

### Importing packages

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import random
from typing import List
from datetime import datetime

import feast
import pandas as pd
from elemeno_ai_sdk.ml.features.feature_store import FeatureStore
from elemeno_ai_sdk.ml.features.feature_table import FeatureTable
from elemeno_ai_sdk.ml.features.ingest.sink.ingestion_sink_builder import IngestionSinkType

In [ ]:
os.environ["AWS_ACCESS_KEY_ID"] = "insert your access key id here"
os.environ["AWS_SECRET_ACCESS_KEY"] = "insert your secret access key here"

### Dataframe to ingest

In [ ]:
FILE_PATH = "./example_data/datasource.csv"
data = pd.read_csv(FILE_PATH)

In [ ]:
data.head()

### Creating feature store

Instantiate the Feature Store object, since we are working with a csv file we do not need to pass a `source_type` into the constructor, we just need to pass the `sink_type` which in our case will be `REDSHIFT`.

In [ ]:
feature_store = FeatureStore(
    sink_type=IngestionSinkType.REDSHIFT
)

Now we create a Feature Table object. First I parse the id and feature columns to feast `Entity` and `Feature` objects:

In [ ]:
def get_entities(id_columns: List[str]) -> List[feast.Entity]:
    return [feast.Entity(name=id_col) for id_col in id_columns] 

def get_features(feature_list: List[str]) -> List[feast.Feature]:
    features = []
    for feature in feature_list:
        if feature == "created_timestamp" or feature == "event_timestamp":
            dtype = feast.ValueType.BYTES
        elif feature == "target":
            dtype = feast.ValueType.INT32
        else:
            dtype = feast.ValueType.FLOAT
        features.append(feast.Feature(name=feature, dtype=dtype))
    return features

In [ ]:
FEATURE_TABLE_NAME = "one_blinc_ft"
FEATURES = [col for col in data.columns if col != "id"]
IDS = ["id"]

In [ ]:
feature_table = FeatureTable(
    name=FEATURE_TABLE_NAME,
    feature_store=feature_store,
    entities=get_entities(id_columns=IDS),
    features=get_features(feature_list=FEATURES),
    online=True
)

### Ingest features

To ingest features we call the `ingest` method and we need to pass the `feature_table` we just created and the `dataframe` we want to save. 

We can also pass two more additional params, `renames` which will rename the features and `all_columns` which will filtered the features from your data source before sending to the feature store, for the purpose of our example we will leave it as ***None***

In [ ]:
feature_store.ingest(
    feature_table=feature_table, 
    to_ingest=data, 
    renames=None, 
    all_columns=None
)

### Retrieve features

In [ ]:
retrieved_data = feature_store.get_training_features(feature_table=feature_table)

In [ ]:
retrieved_data